In [24]:

import pandas as pd
import numpy as np
import re
import math
import os
import subprocess

# Text and feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

# Evaluation and tuning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, roc_curve, auc)

# Classifier
from sklearn.naive_bayes import GaussianNB

# Text cleaning & stopwords
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import torch
import torch.nn
import importlib

[nltk_data] Downloading package stopwords to /Users/dwika/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
import simple_mlp
import TextDataset
importlib.reload(simple_mlp)

<module 'simple_mlp' from '/Volumes/Dwika/ISE/ISE-solution/lab1/simple_mlp.py'>

In [31]:
from torch.utils.data import random_split, DataLoader

# Step 1: Create the full dataset
full_dataset = TextDataset.TextDatasetTFIDF('datasets/pytorch.csv')

# Step 2: Define train/val split sizes
val_ratio = 0.2
val_size = int(len(full_dataset) * val_ratio)
train_size = len(full_dataset) - val_size

# Step 3: Split the dataset
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

Preprocessed DataFrame:
                                                   text  sentiment
584  undefined symbol: cudnnSetConvolutionGroupCoun...          0
591  Unable to build PyTorch from source without Nu...          0
486  [docs] Make clear the format of torch.eig eige...          0
77   `index_select` on flat tensor faster than inte...          1
212  test_det_logdet_slogdet_batched (in test_cuda....          0


In [32]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score, accuracy_score
import helper

In [33]:
model = simple_mlp.MLPWithLayerNorm(input_dim=1000)  # TF-IDF has 1000 features
criterion = nn.CrossEntropyLoss()  # You can add weights here if needed
optimizer = optim.Adam(model.parameters(), lr=0.01)  # Lowered LR
model

MLPWithLayerNorm(
  (fc1): Linear(in_features=1000, out_features=64, bias=True)
  (norm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=2, bias=True)
)

In [34]:
accuracy_scores=[]
f1_scores = []
precision_scores = []
recall_scores = []
auc_scores=[]
epochs=100* len(train_loader)
# ========== Training Loop ==========
for epoch in range(epochs):
    model.train()
    total_train_loss = 0

    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_train_loss += loss.item()

    # Validation
    model.eval()
    all_preds, all_probs, all_targets = [], [], []

    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_targets.extend(batch_y.cpu().numpy())

    # Metrics
    acc = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='binary')
    precision = precision_score(all_targets, all_preds, average='binary')
    recall = recall_score(all_targets, all_preds, average='binary')
    try:
        auc = roc_auc_score(all_targets, all_probs)
        auc_scores.append(auc)
    except ValueError:
        auc = None

    # Save metrics
    accuracy_scores.append(acc)
    f1_scores.append(f1)
    precision_scores.append(precision)
    recall_scores.append(recall)
 


    # print(f"Epoch [{epoch+1}/{epochs}]")
    # print(f"  Train Loss : {total_train_loss:.4f}")
    # print(f"  Val Acc    : {acc * 100:.2f}%")
    # print(f"  Val F1     : {f1:.4f}")
    # print(f"  Val Prec   : {precision:.4f}")
    # print(f"  Val Recall : {recall:.4f}")
    # print(f"  Val AUC    : {auc:.4f}" if auc is not None else "  Val AUC    : Not computable")
    # print("-" * 40)

# ========== Averages ==========
avg_acc = sum(accuracy_scores) / len(accuracy_scores)
avg_f1 = sum(f1_scores) / len(f1_scores)
avg_precision = sum(precision_scores) / len(precision_scores)
avg_recall = sum(recall_scores) / len(recall_scores)
auc_score=sum(auc_scores) / len(auc_scores)

print("\n===== Average Validation Metrics Across Epochs =====")
print(f"Average Accc     : {avg_acc:.4f}")
print(f"Average F1 Score     : {avg_f1:.4f}")
print(f"Average Precision    : {avg_precision:.4f}")
print(f"Average Recall       : {avg_recall:.4f}")
print(f"Average Auc Score     : {auc_score:.4f}")

/Users/dwika/miniconda3/envs/fyp/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


KeyboardInterrupt: 

In [ ]:
import helper
helper.evaluate_model(model,val_loader,"pytorch","pytorch",0)

{'iteration': 0,
 'name': 'pytorch->pytorch',
 'accuracy': 0.9,
 'precision': 0.375,
 'recall': 0.5454545454545454,
 'f1': 0.4444444444444444,
 'auc': 0.9005886200130805}

# Prediction on Different Data Set


In [ ]:
inference_dataset=TextDataset.TextDatasetTFIDF("datasets/caffe.csv")
inference_loader = DataLoader(inference_dataset, batch_size=32, shuffle=False)

Preprocessed DataFrame:
                                                   text  sentiment
9    unable to reproduce accuracy of bvlc-alexnet. ...          1
267  osx: abs not defined absval_layer. When compil...          0
143  cafe_intsall.caffe 36 error. I am trying caffe...          0
212   undefined reference to `lzma_index_end@XZ_5.0...          0
227  Dimension mismatch training with my own model ...          0


In [ ]:
model.eval()
all_preds = []
all_probs = []
all_targets = []

with torch.no_grad():
    for batch_X, batch_y in inference_loader:
        outputs = model(batch_X)
        probs = torch.softmax(outputs, dim=1)
        preds = torch.argmax(probs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs[:, 1].cpu().numpy())
        all_targets.extend(batch_y.cpu().numpy())


In [ ]:
accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average='binary')
recall = recall_score(all_targets, all_preds, average='binary')
f1 = f1_score(all_targets, all_preds, average='binary')
try:
    auc = roc_auc_score(all_targets, all_probs)
except ValueError:
    auc = None

print(f"Accuracy  : {accuracy:.4f}")
print(f"Precision : {precision:.4f}")
print(f"Recall    : {recall:.4f}")
print(f"F1 Score  : {f1:.4f}")
print(f"AUC       : {auc:.4f}" if auc is not None else "AUC: Not computable")

Accuracy  : 0.8601
Precision : 0.1818
Recall    : 0.0606
F1 Score  : 0.0909
AUC       : 0.5970


# Trying Tent on Title Body Caffe

In [ ]:
import tent
import TextDataset
import importlib
# import batch_data_loader
importlib.reload(TextDataset)  
importlib.reload(tent)
import copy

In [ ]:
model_state = copy.deepcopy(model.state_dict())

In [4]:
model=tent.configureTent(model)
result=tent.Tent(model,inference_dataset,"pytorch","caffe",1)
print(result)
tent.reset(model,model_state)

NameError: name 'tent' is not defined